In [1]:
import pandas as pd 
import numpy as np

In [2]:
#Choix des coefficients

coeff_entité = 0.1
coeff_délai = 1
coeff_consultant = 2
coeff_langue = 1.5
coeff_salaire = 1
coeff_distance = 1

In [3]:
df = pd.read_excel("Appel d'offre.xlsx")
df

,Poste,Lieu,Experience,Entité,Délai,Nb_Consultant,Salaire,Compétence
0,DATA,"Quai Saint-Exupéry, 92100 Paris",1.0,Microsoft,16.0,1.0,3000.0,SCALA;HADOOP;PYTHON;AZURE;ELK
1,JAVA,"3 Boulevard Ney, 75018 Paris",5.0,Amazon,4.0,4.0,4500.0,JAVA;C++;PYTHON;AWS;SQL;JAVASCRIPT;HIBERNATE
2,RH,"8 Rue de Londres, 75009 Paris",3.0,Google,7.0,7.0,2700.0,OFFICE365;ELK;HADOOP
3,CLOUD,"6 Rue Ménars, 75002 Paris",0.0,Alliexpress,31.0,0.0,2500.0,SPARK;HTML;DJANGO;FLASK
4,UNIX,"12 Avenue Pierre Mendès-France, 75013 Paris",10.0,Carrefour,7.0,11.0,2300.0,JAVASCRIPT;LINUX
5,NaN,78 rue Olivier de Serres - 75015 Paris 15e arr...,NaN,Orange,NaN,NaN,NaN,NaN


In [4]:
lang = df['Compétence'].str.get_dummies(sep=';')
df = df.drop('Compétence',axis=1)

In [5]:
df['Entité']= df['Entité'].apply(lambda x : x.upper())

In [6]:
companies = pd.read_csv('companies_ranking.csv')
companies['Entité']= companies['Entité'].apply(lambda x : x.upper())
mask = []
for i in range(len(df)):
    if df.iloc[i]['Entité'] in companies.Entité.values:
        mask.append(df.iloc[i]['Entité'])
        #pd.concat(df.iloc[i],df[df['Entité']==df.iloc[i]['Entité']])
        
companies = companies.loc[companies['Entité'].isin(mask)]
companies

,Valeur,Entité
0,99,MICROSOFT
19,80,GOOGLE


In [30]:
CAC40 = pd.read_csv('preprocessed_CAC40.csv',sep=',')
CAC40 = CAC40.groupby('Name').sum('Volume').reset_index()
CAC40.rename(columns = {'Unnamed: 0' : 'Valeur', 'Name' : 'Entité'},inplace = True)
CAC40 = CAC40.sort_values('Valeur', ascending = False)
CAC40 = CAC40[['Entité']]
CAC40['Entité']= CAC40['Entité'].apply(lambda x : x.upper())
CAC40.reset_index(inplace = True)
CAC40.rename(columns = {'index' : 'Valeur'},inplace = True)

In [33]:
for i in range(len(df)):
    if df.iloc[i]['Entité'] in CAC40.Entité.values:
        mask.append(df.iloc[i]['Entité'])
        #pd.concat(df.iloc[i],df[df['Entité']==df.iloc[i]['Entité']])
        
CAC40 = CAC40.loc[CAC40['Entité'].isin(mask)]
CAC40

,Valeur,Entité
17,20,ORANGE


In [34]:
mask

['MICROSOFT', 'GOOGLE', 'ORANGE', 'ORANGE', 'ORANGE', 'ORANGE']

In [11]:
CAC40.Entité.values

array(['VIVENDI', 'VINCI', 'VEOLIA ENVIRONNEMENT', 'UNIBAIL-RODAMCO',
       'TOTAL', 'STMICROELECTRONICS', 'SODEXO',
       'SOCIÉTÉ GÉNÉRALE (SOCIETE GENERALE', 'SCHNEIDER ELECTRIC',
       'SANOFI', 'SAINT-GOBAIN', 'SAFRAN', 'RENAULT', 'PUBLICIS',
       'PEUGEOT', 'PERNOD RICARD', 'WORLDLINE SA', 'ORANGE',
       'MICHELIN (COMPAGNIE GÉNÉRALE D ETABLISSEMENTS MICHELIN SCPA',
       'LVMH MOET HENNESSY LOUIS VUITTON', 'LORÉAL', 'LEGRAND', 'KERING',
       'HERMÈS (HERMES INTERNATIONAL', 'ESSILORLUXOTTICA',
       'ENGIE (EX GDF SUEZ', 'DASSAULT SYSTÈMES', 'DANONE',
       'CRÉDIT AGRICOLE', 'CAP GEMINI', 'BOUYGUES', 'BNP PARIBAS', 'AXA',
       'ATOS', 'ARCELORMITTAL', 'AIRBUS ', 'AIR LIQUIDE', 'ACCOR'],
      dtype=object)

In [9]:
for i in range(len(df)):
    if df.iloc[i]['Entité'] in companies.Entité.values:
        df['Entité'][i] = int(companies.loc[companies['Entité']==df.iloc[i]['Entité']]['Valeur'].values)
    elif df.iloc[i]['Entité'] in CAC40.Entité.values:
        df['Entité'][i] = int(companies.loc[companies['Entité']==df.iloc[i]['Entité']]['Valeur'].values)
    else :
        df['Entité'][i] = 0
        


C:\Users\Utilisateur\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Utilisateur\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


TypeError: only size-1 arrays can be converted to Python scalars

In [ ]:
df['Entité']

In [ ]:
def con_Scoring(x):
    if (x > 10) : 
        x = 1
    elif (x > 5) :
        x = 0.66 
    elif (x > 0) :
        x = 0.33
    else :
        x = 0
    return x

df['Nb_Consultant'] = df['Nb_Consultant'].apply(lambda x : con_Scoring(x))
df['Nb_Consultant']

In [ ]:
def exp_Scoring(x):
    if (x > 10) : 
        x = 1
    elif (x > 5) :
        x = 0.75
    elif (x > 2) :
        x = 0.5 
    elif (x > 0) :
        x = 0.25
    else :
        x = 0
    return x

df['Experience'] = df['Experience'].apply(lambda x : exp_Scoring(x))
df['Experience']

In [ ]:
def sal_Scoring(x):
    if (x > 5000) : 
        x = 1
    elif (x > 3500) :
        x = 0.66
    elif (x > 2000) :
        x = 0.33
    else :
        x = 0
    return x

df['Salaire'] = df['Salaire'].apply(lambda x : sal_Scoring(x))
df['Salaire']

In [ ]:
poste = df[['Poste']]
df.drop('Poste',axis=1,inplace=True)

Lieu = df[['Lieu']]
df.drop('Lieu',axis=1,inplace=True)

In [ ]:
normalized_df=(df-df.min())/(df.max()-df.min())
normalized_df

df = poste.join(normalized_df)

In [ ]:
comp_E = []
comp_NE = []


data_comp_E = ['PYTHON','R', 'SCALA','C++','HADOOP','SPARK','JAVASCRIPT','DOCKER']
data_comp_NE = ['HIVE','HBASE','MESOS','COUCHBASE', 'ELK','KUBERNETES','SQL']

java_comp_E = ['JAVA', 'J2EE', 'SPRING', 'SPRING BOOT', 'REST']
java_comp_NE = ['HIBERNATE', 'GROOVY', 'GRAILS', 'ANGULAR', 'IAAS', 'AWS', 'ELK']

devops_comp_E = ['IAC','PACKER', 'ANSIBLE', 'SALTSTACK', 'PUPPET', 'CHEF','DOCKER','RANCHEROS','PYTHON','BASH','SHELL']
devops_comp_NE = ['JENKINS','Gitlab-CI', 'Travis-CI', 'XL-Deploy','KUBERNETES','AWS']


In [ ]:
def lang_scoring(i,e,ne):
    for x in lang:
        
            if x in e:
                lang.iloc[i][x] = lang.iloc[i][x]*2
            elif x in ne:
                lang.iloc[i][x] = lang.iloc[i][x]*1
            else :
                lang.iloc[i][x] = 0

In [ ]:
for i in range(0,len(df)):
    if df.iloc[i]['Poste'] == 'DATA':
        lang_scoring(i,data_comp_E,data_comp_NE)
    elif df.iloc[i]['Poste'] == 'JAVA':
        lang_scoring(i,java_comp_E,java_comp_NE)
    elif df.iloc[i]['Poste'] == 'DEVOPS':
        lang_scoring(i,devops_comp_E,devops_comp_NE)
    else :
        lang_scoring(i,comp_E,comp_NE)

In [ ]:
lang['lang_Score'] = lang.sum(axis=1)
lang

In [ ]:
Score = lang['lang_Score']
Score =(Score-Score.min())/(Score.max()-Score.min())

In [ ]:
normalized_df = normalized_df.join(Score)
normalized_df

In [ ]:
import requests, json
import urllib.parse
from math import pi,sqrt,sin,cos,atan2

api_url = "https://api-adresse.data.gouv.fr/search/?q="

def get_add(x):
    adr = x
    r = requests.get(api_url + urllib.parse.quote(adr))
    data = json.loads(r.content)
    
    return data.get('features')[0].get("geometry").get('coordinates')

Lieu['coord'] = Lieu['Lieu'].apply(lambda x : get_add(x))

In [ ]:
add_op = '29 Rue des Sablons, 75116 Paris'

add_op = get_add(add_op)
lat_op = add_op[1]
lon_op = add_op[0]

from math import sin, cos, acos, pi
 
def distanceGPS(longB, latB):
    
    degree_to_rad = float(pi / 180.0)
    
    d_lat = (latB - lat_op) * degree_to_rad
    d_long = (longB - lon_op) * degree_to_rad
    a = pow(sin(d_lat / 2), 2) + cos(lat_op * degree_to_rad) * cos(latB * degree_to_rad) * pow(sin(d_long / 2), 2)
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    km = 6367 * c
    return km
 
Lieu['distance'] = Lieu['coord'].apply(lambda x : distanceGPS(x[0],x[1]))

In [ ]:
def dist_score(x):
    if (x > 30) : 
        x = 0.33
    elif (x > 15) :
        x = 0.66 
    elif (x <= 15) :
        x = 1
    else :
        x = 0
    return x


Lieu['dist_score'] = Lieu['distance'].apply(lambda x : dist_score(x))
dist_score = Lieu[['dist_score']]
normalized_df = normalized_df.join(dist_score)

In [ ]:
def Exp_coeff(x):
    x = x*coeff_experience
    return x    
def En_coeff(x):
    x = x*coeff_entité
    return x
def Del_coeff(x):
    x= x*coeff_délai
    return x
def Con_coeff(x):
    x= x*coeff_consultant
    return x
def Sal_coeff(x):
    x = x*coeff_salaire
    return x
def Lang_coeff(x):
    x = x*coeff_langue
    return x
def Dist_coeff(x):
    x = x*coeff_distance
    return x

def get_score(x):
    x['Entité'] = x['Entité'].apply(lambda x : En_coeff(x))
    x['Nb_Consultant'] =  x['Nb_Consultant'].apply(lambda x : Con_coeff(x))
    x['lang_Score'] =  x['lang_Score'].apply(lambda x : Lang_coeff(x))
    x['dist_score'] =  x['dist_score'].apply(lambda x : Dist_coeff(x))


In [ ]:
get_score(normalized_df)
normalized_df

In [ ]:
normalized_df['Score'] = normalized_df.sum(axis=1)
normalized_df.sort_values(by='Score',ascending=False)